# Quiver Movie

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import dustpy
from plot_helper import plot_quiver, get_transparent_cmap, num2tex

In [ ]:
reader = dustpy.hdf5writer()
reader.datadir = 'data_bump/'
files = reader.read.listfiles()

## Make the frames & compile movie

In [ ]:
dirname = Path('frames')
dirname.mkdir(exist_ok=True)

for i, file in tqdm(enumerate(files), total=len(files)):
    f, ax = plot_quiver(file)
    ax.set_facecolor('none')
    #f.savefig(dirname / f'frame_{i:03d}.jpg', transparent=False, dpi=300)
    f.savefig(dirname / f'frame_{i:03d}.png', transparent=True, dpi=300)
    plt.close(f)

In [ ]:
!ffmpeg -y -r 12 -i {dirname}/frame_%03d.png -c:v prores_ks -profile:v 5 -vendor apl0  -bits_per_mb 8000 -pix_fmt yuva444p10le out.mov

In [ ]:
!open out.mov

## Make a PDF

In [ ]:
f, ax = plot_quiver(file)
ax.set_facecolor('none')
f.savefig(f'frame_{i:03d}.pdf', transparent=True)